In [1]:
import pandas as pd
import numpy as np
from scipy import interpolate
from src.online_learning.cluster import Cluster

In [2]:
# Load the Excel file with all sheets
file_path1 = 'initialization/DT_CU1_parameters_1_exp.xlsx'  
all_sheets = pd.read_excel(file_path1, sheet_name=None)

# Check the sheet names
print(all_sheets.keys())

dict_keys(['T = 25', 'T = 10', 'T = 40'])


In [3]:
df_list = []
for sheet_name, df in all_sheets.items():
    print(f"Sheet name: {sheet_name}")
    df_list.append(df)

Sheet name: T = 25
Sheet name: T = 10
Sheet name: T = 40


In [4]:
df_T_25 = df_list[0]
df_T_10 = df_list[1]
df_T_40 = df_list[2]

In [5]:
columns_to_drop = ['filename', 'data', 'start interval', 'end interval',
                   'tau_guess','num value extrap',
                   'R squared', 'OCV', 'tau_opt']

In [6]:
df_T_25 = df_T_25.drop(columns_to_drop, axis=1)
df_T_10 = df_T_10.drop(columns_to_drop, axis=1)
df_T_40 = df_T_40.drop(columns_to_drop, axis=1)

In [7]:
df_T_25 = df_T_25.dropna()
df_T_10 = df_T_10.dropna()
df_T_40 = df_T_40.dropna()

In [8]:
df_T_25['Temp'] = 25
df_T_10['Temp'] = 10
df_T_40['Temp'] = 40

In [9]:
df_T_25.describe()

,Soc,R_0,R_1,C_1,Temp
count,55.000000,55.000000,55.000000,55.000000,55.0
mean,50.000000,0.011520,0.015499,11965.948149,25.0
std,31.914237,0.002412,0.006471,3143.296922,0.0
min,0.000000,0.009210,0.009238,4919.388300,25.0
25%,20.000000,0.010363,0.011560,10500.078051,25.0
50%,50.000000,0.010981,0.013876,11751.825878,25.0
75%,80.000000,0.011748,0.016568,13995.291319,25.0
max,100.000000,0.022375,0.041071,19489.447086,25.0


In [10]:
df_T_25.describe()

,Soc,R_0,R_1,C_1,Temp
count,55.000000,55.000000,55.000000,55.000000,55.0
mean,50.000000,0.011520,0.015499,11965.948149,25.0
std,31.914237,0.002412,0.006471,3143.296922,0.0
min,0.000000,0.009210,0.009238,4919.388300,25.0
25%,20.000000,0.010363,0.011560,10500.078051,25.0
50%,50.000000,0.010981,0.013876,11751.825878,25.0
75%,80.000000,0.011748,0.016568,13995.291319,25.0
max,100.000000,0.022375,0.041071,19489.447086,25.0


In [13]:
df_T_10_soc_one = df_T_10[( df_T_10['Soc'] >= 0) & (df_T_10['Soc'] <= 50) ]
df_T_25_soc_one = df_T_25[ ( df_T_25['Soc'] >= 0) & (df_T_25['Soc'] <= 50) ]
df_T_40_soc_one = df_T_40[ ( df_T_40['Soc'] >= 0) & (df_T_40['Soc'] <= 50) ]

chunk_size = 6
temp = np.array([10, 25])
soc = np.array([0, 10, 20, 30, 40, 50])
parameters = ['R_0', 'R_1', 'C_1']
temp_new = [15, 20]

# List to store results for each parameter
interpolated_results = []

for start in range(0, len(df_T_10_soc_one), chunk_size):
    chunk_10 = df_T_10_soc_one.iloc[start:start + chunk_size]
    chunk_25 = df_T_25_soc_one.iloc[start:start + chunk_size]
    
    for param in parameters:
        # Extracting the parameter values for the chunks
        values_10 = chunk_10[param].values
        values_25 = chunk_25[param].values
        
        # Check the shape of values_10 and values_25 to make sure they match soc.size
        if len(values_10) != len(soc) or len(values_25) != len(soc):
            print(f"Chunk size mismatch for parameter {param} at index {start}")
            continue
        
        # Creating the array for interpolation with the correct shape (n, m)
        z = np.vstack((values_10, values_25)).T  # Transpose to get shape (n, m)
        
        # Performing the interpolation
        f = interpolate.interp2d(temp, soc, z, kind='linear')
        z_new = f(temp_new, soc)
        
        # Store the interpolated results
        for i, temp_val in enumerate(temp_new):
            for j, soc_val in enumerate(soc):
                interpolated_results.append({
                    'Temp': temp_val,
                    'Soc': soc_val,
                    'param': param,
                    'value': z_new[j, i]
                })

# Convert the interpolated results to a DataFrame
df_interpolated = pd.DataFrame(interpolated_results)

# Pivot the DataFrame to get the desired structure
df_interpolated_pivot = df_interpolated.pivot_table(index=['Temp', 'Soc'], columns='param', values='value').reset_index()

print(df_interpolated_pivot)

param  Temp  Soc           C_1       R_0       R_1
0        15    0   9362.838793  0.016177  0.016432
1        15   10   9691.182929  0.016623  0.019499
2        15   20  12052.351794  0.017420  0.023310
3        15   30   9306.629723  0.018662  0.024055
4        15   40   5139.236488  0.024588  0.035098
5        15   50   2142.033750  0.025116  0.108859
6        20    0  11140.809709  0.013595  0.014089
7        20   10  11096.331372  0.014090  0.016753
8        20   20  13364.479790  0.014450  0.019963
9        20   30  11500.390560  0.015292  0.019551
10       20   40   8387.385488  0.018321  0.025878
11       20   50   4165.597126  0.020244  0.070274


/tmp/ipykernel_9666/1338578928.py:32: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://gist.github.com/ev-br/8544371b40f414b7eaf3fe6217209bff`

  f = interpolate.interp2d(temp, soc, z, kind='linear')
/tmp/ipykernel_9666/1338578928.py:33: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
        scattered 2D data.

        In new code, for regular gr

In [14]:
df_interpolated_pivot.describe()

param,Temp,Soc,C_1,R_0,R_1
count,12.000000,12.000000,12.000000,12.000000,12.000000
mean,17.500000,25.000000,8945.772293,0.017882,0.032814
std,2.611165,17.837652,3437.060362,0.003809,0.028287
min,15.000000,0.000000,2142.033750,0.013595,0.014089
25%,15.000000,10.000000,7575.348238,0.015082,0.018813
50%,17.500000,25.000000,9527.010861,0.017022,0.021636
75%,20.000000,40.000000,11230.704922,0.019057,0.028183
max,20.000000,50.000000,13364.479790,0.025116,0.108859


In [60]:
cluster_one = pd.concat([df_interpolated_pivot,df_T_10_soc_one,df_T_25_soc_one])

In [61]:
cluster_one = cluster_one.drop('skip', axis=1)

In [63]:
cluster_one.to_csv("phi_one.csv", index=False, na_rep='NaN')

## cell2

In [34]:
df_T_10_soc_two = df_T_10[( df_T_10['Soc'] >= 50) & (df_T_10['Soc'] <= 100) ]
df_T_25_soc_two = df_T_25[ ( df_T_25['Soc'] >= 50) & (df_T_25['Soc'] <= 100) ]
#df_T_40_soc_one = df_T_40[ ( df_T_40['Soc'] >= 0) & (df_T_40['Soc'] <= 50) ]

chunk_size = 6
temp = np.array([10, 25])
soc = np.array([0, 10, 20, 30, 40, 50])
parameters = ['R_0', 'R_1', 'C_1']
temp_new = [15, 20]

# List to store results for each parameter
interpolated_results = []

for start in range(0, len(df_T_10_soc_two), chunk_size):
    chunk_10 = df_T_10_soc_two.iloc[start:start + chunk_size]
    chunk_25 = df_T_25_soc_one.iloc[start:start + chunk_size]
    
    for param in parameters:
        # Extracting the parameter values for the chunks
        values_10 = chunk_10[param].values
        values_25 = chunk_25[param].values
        
        # Check the shape of values_10 and values_25 to make sure they match soc.size
        if len(values_10) != len(soc) or len(values_25) != len(soc):
            print(f"Chunk size mismatch for parameter {param} at index {start}")
            continue
        
        # Creating the array for interpolation with the correct shape (n, m)
        z = np.vstack((values_10, values_25)).T  # Transpose to get shape (n, m)
        
        # Performing the interpolation
        f = interpolate.interp2d(temp, soc, z, kind='linear')
        z_new = f(temp_new, soc)
        
        # Store the interpolated results
        for i, temp_val in enumerate(temp_new):
            for j, soc_val in enumerate(soc):
                interpolated_results.append({
                    'Temp': temp_val,
                    'Soc': soc_val,
                    'param': param,
                    'value': z_new[j, i]
                })

# Convert the interpolated results to a DataFrame
df_interpolated_two = pd.DataFrame(interpolated_results)

# Pivot the DataFrame to get the desired structure
df_interpolated_pivot_two = df_interpolated_two.pivot_table(index=['Temp', 'Soc'], columns='param', values='value').reset_index()

print(df_interpolated_pivot_two)

param  Temp  Soc           C_1       R_0       R_1
0        15    0   9655.887330  0.013612  0.014383
1        15   10   8960.577385  0.014149  0.014264
2        15   20   9455.065286  0.014868  0.018994
3        15   30  14273.722858  0.015673  0.021401
4        15   40   8889.953426  0.015858  0.017680
5        15   50   7119.632085  0.017630  0.023079
6        20    0  11287.333977  0.012312  0.013065
7        20   10  10731.028600  0.012853  0.014136
8        20   20  12065.836535  0.013174  0.017804
9        20   30  13983.937127  0.013798  0.018224
10       20   40  10262.743957  0.013956  0.017169
11       20   50   6654.396293  0.016500  0.027384


/tmp/ipykernel_9666/1730702352.py:32: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://gist.github.com/ev-br/8544371b40f414b7eaf3fe6217209bff`

  f = interpolate.interp2d(temp, soc, z, kind='linear')
/tmp/ipykernel_9666/1730702352.py:33: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
        scattered 2D data.

        In new code, for regular gr

In [37]:
cluster_two = pd.concat([df_interpolated_pivot_two,df_T_10_soc_two,df_T_25_soc_two])

In [38]:
len(cluster_two)

60

In [42]:
cluster_two = cluster_two.drop('skip', axis=1)

In [43]:
cluster_two.describe()

,Temp,Soc,C_1,R_0,R_1
count,60.000000,60.000000,60.000000,60.000000,60.000000
mean,19.000000,65.000000,10898.610389,0.013090,0.015514
std,6.689227,26.521498,3296.695650,0.003160,0.004286
min,10.000000,0.000000,6329.622654,0.009210,0.009238
25%,10.000000,50.000000,8070.132353,0.010474,0.011761
50%,22.500000,70.000000,10351.731264,0.011748,0.014324
75%,25.000000,90.000000,13357.329312,0.015858,0.018259
max,25.000000,100.000000,19489.447086,0.019599,0.027384


In [44]:
cluster_two.to_csv("phi_two.csv", index=False, na_rep='NaN')

## cell3

In [46]:
#df_T_10_soc_one = df_T_10[( df_T_10['Soc'] >= 0) & (df_T_10['Soc'] <= 50) ]
df_T_25_soc_three = df_T_25[ ( df_T_25['Soc'] >= 0) & (df_T_25['Soc'] <= 50) ]
df_T_40_soc_three = df_T_40[ ( df_T_40['Soc'] >= 0) & (df_T_40['Soc'] <= 50) ]

chunk_size = 6
temp = np.array([25, 40])
soc = np.array([0, 10, 20, 30, 40, 50])
parameters = ['R_0', 'R_1', 'C_1']
temp_new = [30, 35]

# List to store results for each parameter
interpolated_results = []

for start in range(0, len(df_T_10_soc_one), chunk_size):
    chunk_25 = df_T_25_soc_one.iloc[start:start + chunk_size]
    chunk_40 = df_T_40_soc_one.iloc[start:start + chunk_size]
    
    for param in parameters:
        # Extracting the parameter values for the chunks
        values_25 = chunk_25[param].values
        values_40 = chunk_40[param].values
        
        # Check the shape of values_10 and values_25 to make sure they match soc.size
        if len(values_25) != len(soc) or len(values_40) != len(soc):
            print(f"Chunk size mismatch for parameter {param} at index {start}")
            continue
        
        # Creating the array for interpolation with the correct shape (n, m)
        z = np.vstack((values_25, values_40)).T  # Transpose to get shape (n, m)
        
        # Performing the interpolation
        f = interpolate.interp2d(temp, soc, z, kind='linear')
        z_new = f(temp_new, soc)
        
        # Store the interpolated results
        for i, temp_val in enumerate(temp_new):
            for j, soc_val in enumerate(soc):
                interpolated_results.append({
                    'Temp': temp_val,
                    'Soc': soc_val,
                    'param': param,
                    'value': z_new[j, i]
                })

# Convert the interpolated results to a DataFrame
df_interpolated_three = pd.DataFrame(interpolated_results)

# Pivot the DataFrame to get the desired structure
df_interpolated_pivot_three = df_interpolated.pivot_table(index=['Temp', 'Soc'], columns='param', values='value').reset_index()

print(df_interpolated_pivot_three)

param  Temp  Soc           C_1       R_0       R_1
0        15    0   9362.838793  0.016177  0.016432
1        15   10   9691.182929  0.016623  0.019499
2        15   20  12052.351794  0.017420  0.023310
3        15   30   9306.629723  0.018662  0.024055
4        15   40   5139.236488  0.024588  0.035098
5        15   50   2142.033750  0.025116  0.108859
6        20    0  11140.809709  0.013595  0.014089
7        20   10  11096.331372  0.014090  0.016753
8        20   20  13364.479790  0.014450  0.019963
9        20   30  11500.390560  0.015292  0.019551
10       20   40   8387.385488  0.018321  0.025878
11       20   50   4165.597126  0.020244  0.070274


/tmp/ipykernel_9666/2542729913.py:32: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://gist.github.com/ev-br/8544371b40f414b7eaf3fe6217209bff`

  f = interpolate.interp2d(temp, soc, z, kind='linear')
/tmp/ipykernel_9666/2542729913.py:33: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
        scattered 2D data.

        In new code, for regular gr

In [47]:
cluster_three = pd.concat([df_interpolated_pivot_three, df_T_40_soc_three])

In [48]:
len(cluster_three)

30

In [50]:
cluster_three = cluster_three.drop('skip', axis=1)

In [51]:
cluster_three.to_csv("phi_three.csv", index=False, na_rep='NaN')

## cell4

In [52]:
#df_T_10_soc_two = df_T_10[( df_T_10['Soc'] >= 50) & (df_T_10['Soc'] <= 100) ]
df_T_25_soc_four = df_T_25[ ( df_T_25['Soc'] >= 50) & (df_T_25['Soc'] <= 100) ]
df_T_40_soc_four = df_T_40[ ( df_T_40['Soc'] >= 50) & (df_T_40['Soc'] <= 100) ]

chunk_size = 6
temp = np.array([25, 40])
soc = np.array([0, 10, 20, 30, 40, 50])
parameters = ['R_0', 'R_1', 'C_1']
temp_new = [30, 35]

# List to store results for each parameter
interpolated_results = []

for start in range(0, len(df_T_10_soc_two), chunk_size):
    chunk_25 = df_T_25_soc_four.iloc[start:start + chunk_size]
    chunk_40 = df_T_40_soc_four.iloc[start:start + chunk_size]
    
    for param in parameters:
        # Extracting the parameter values for the chunks
        values_25 = chunk_25[param].values
        values_40 = chunk_40[param].values
        
        # Check the shape of values_10 and values_25 to make sure they match soc.size
        if len(values_25) != len(soc) or len(values_40) != len(soc):
            print(f"Chunk size mismatch for parameter {param} at index {start}")
            continue
        
        # Creating the array for interpolation with the correct shape (n, m)
        z = np.vstack((values_25, values_40)).T  # Transpose to get shape (n, m)
        
        # Performing the interpolation
        f = interpolate.interp2d(temp, soc, z, kind='linear')
        z_new = f(temp_new, soc)
        
        # Store the interpolated results
        for i, temp_val in enumerate(temp_new):
            for j, soc_val in enumerate(soc):
                interpolated_results.append({
                    'Temp': temp_val,
                    'Soc': soc_val,
                    'param': param,
                    'value': z_new[j, i]
                })

# Convert the interpolated results to a DataFrame
df_interpolated_four = pd.DataFrame(interpolated_results)

# Pivot the DataFrame to get the desired structure
df_interpolated_pivot_four = df_interpolated_two.pivot_table(index=['Temp', 'Soc'], columns='param', values='value').reset_index()

print(df_interpolated_pivot_four)

param  Temp  Soc           C_1       R_0       R_1
0        15    0   9655.887330  0.013612  0.014383
1        15   10   8960.577385  0.014149  0.014264
2        15   20   9455.065286  0.014868  0.018994
3        15   30  14273.722858  0.015673  0.021401
4        15   40   8889.953426  0.015858  0.017680
5        15   50   7119.632085  0.017630  0.023079
6        20    0  11287.333977  0.012312  0.013065
7        20   10  10731.028600  0.012853  0.014136
8        20   20  12065.836535  0.013174  0.017804
9        20   30  13983.937127  0.013798  0.018224
10       20   40  10262.743957  0.013956  0.017169
11       20   50   6654.396293  0.016500  0.027384


/tmp/ipykernel_9666/528985934.py:32: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://gist.github.com/ev-br/8544371b40f414b7eaf3fe6217209bff`

  f = interpolate.interp2d(temp, soc, z, kind='linear')
/tmp/ipykernel_9666/528985934.py:33: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
        scattered 2D data.

        In new code, for regular grid

In [53]:
cluster_four = pd.concat([df_interpolated_pivot_four, df_T_40_soc_four])

In [54]:
len(cluster_four)

30

In [55]:
cluster_four = cluster_four.drop('skip', axis =1)

In [56]:
cluster_four.describe()

,Temp,Soc,C_1,R_0,R_1
count,30.000000,30.000000,30.000000,30.000000,30.000000
mean,31.000000,55.000000,13702.497715,0.010468,0.012837
std,11.325893,30.371266,4368.180041,0.003522,0.005340
min,15.000000,0.000000,6654.396293,0.007287,0.006666
25%,20.000000,32.500000,10460.604633,0.007684,0.008535
50%,40.000000,55.000000,13865.795274,0.008014,0.011618
75%,40.000000,80.000000,16267.990572,0.013751,0.016473
max,40.000000,100.000000,27306.615043,0.017630,0.027384


In [58]:
cluster_four.to_csv("phi_four.csv", index=False, na_rep='NaN')